In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [2]:
cleared_data_all = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset.xlsx')
cleared_data_all = cleared_data_all.iloc[:, 1:]
cleared_data_all

,Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (current US$)","Adjusted net savings, including particulate emission damage (current US$)",Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (current US$),...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.484076,0.018315,0.001125,0.493361,0.232096,0.017700,0.025495,0.037758,0.000056,0.000635,...,0.108886,0.023856,0.076645,0.056228,0.279695,0.201838,0.156617,0.323412,0.336784,0.355807
1,0.468048,0.014754,0.001301,0.482257,0.223844,0.014627,0.026590,0.037826,0.000128,0.000285,...,0.080654,0.003790,0.109930,0.043929,0.193565,0.248208,0.176652,0.378192,0.258009,0.438305
2,0.481640,0.017279,0.000530,0.473711,0.250464,0.014431,0.026371,0.038822,0.000118,0.000204,...,0.091330,0.033513,0.087172,0.036492,0.235089,0.298487,0.198736,0.409490,0.283037,0.495832
3,0.453289,0.016555,0.001096,0.499597,0.228257,0.023174,0.026353,0.034463,0.000179,0.000457,...,0.052015,0.022731,0.091999,0.035654,0.245976,0.240291,0.310167,0.292724,0.318325,0.516666
4,0.470644,0.022188,0.000600,0.508247,0.202923,0.022862,0.026775,0.030509,0.000266,0.000555,...,0.044705,0.007773,0.118308,0.038628,0.240441,0.223383,0.363340,0.317195,0.347051,0.452032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.355408,0.021423,0.021071,0.376423,0.394665,0.319516,0.022198,0.021916,0.046591,0.021313,...,0.786150,0.826007,0.640357,0.728138,0.083754,0.051664,0.078857,0.482987,0.446748,0.472202
2028,0.355226,0.015958,0.012176,0.379859,0.443141,0.318189,0.022149,0.022046,0.043437,0.015338,...,0.525959,0.723408,0.744625,0.823676,0.091201,0.047630,0.097655,0.475970,0.344713,0.593125
2029,0.355856,0.020443,0.021131,0.377490,0.433445,0.300040,0.022193,0.021948,0.045275,0.021177,...,0.470119,0.819686,0.710154,0.540721,0.085964,0.065204,0.085008,0.599174,0.528091,0.484817
2030,0.356276,0.017869,0.021781,0.377799,0.392040,0.331834,0.022217,0.021949,0.036685,0.021008,...,0.553846,0.799002,0.719372,0.580822,0.081256,0.063866,0.049129,0.505235,0.456719,0.395095


In [3]:
cleared_data_only_percent = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset_only_%.xlsx')
cleared_data_only_percent = cleared_data_only_percent.iloc[:, 1:]
cleared_data_only_percent

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.457406,0.453062,0.569723,0.657691,0.204321,0.303860,0.342552,0.572192,0.932416,0.255100,...,0.068611,0.047416,0.111070,0.056180,0.229610,0.294200,0.233323,0.343718,0.338674,0.365386
1,0.457128,0.470747,0.605518,0.697599,0.379046,0.277715,0.326866,0.571897,0.970064,0.291585,...,0.067162,0.054393,0.083389,0.069938,0.260012,0.235632,0.294720,0.268808,0.404593,0.345781
2,0.513473,0.452032,0.639689,0.574743,0.386501,0.228170,0.400955,0.559396,0.916655,0.282468,...,0.084913,0.062121,0.088835,0.078727,0.228531,0.239684,0.268762,0.360262,0.354494,0.379005
3,0.524539,0.472299,0.610210,0.573498,0.448555,0.245257,0.407111,0.549325,0.827048,0.278518,...,0.105380,0.039375,0.094489,0.041022,0.202504,0.259351,0.266026,0.298337,0.474548,0.358246
4,0.539012,0.476936,0.619483,0.490571,0.540174,0.403026,0.375958,0.539631,0.742310,0.354712,...,0.105057,0.038862,0.093814,0.024142,0.306630,0.265141,0.229772,0.354893,0.295688,0.201926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.380864,0.344909,0.564977,0.023685,0.367445,0.537930,0.142920,0.388038,0.306280,0.802443,...,0.575320,0.676519,0.781112,0.671105,0.093923,0.071851,0.056182,0.350254,0.506756,0.504535
2028,0.379569,0.343628,0.611090,0.088613,0.421711,0.715505,0.217099,0.386017,0.277552,0.943814,...,0.423149,0.647875,0.941687,0.748946,0.091576,0.075240,0.075146,0.441824,0.619265,0.562003
2029,0.379445,0.345375,0.617299,0.077532,0.320089,0.629314,0.220856,0.357104,0.256545,0.750134,...,0.654935,0.719199,0.776202,0.660155,0.096665,0.074015,0.083483,0.422319,0.487406,0.472707
2030,0.379651,0.341619,0.688184,0.069574,0.398203,0.801984,0.253660,0.367616,0.194880,0.752336,...,0.528392,0.576105,0.519103,0.780148,0.092425,0.074955,0.063195,0.507861,0.401599,0.612767


Division into training and test samples

In [4]:
test_size = 0.2
seed = 7
target_feature_name = 'GDP per capita (current US$)'

data_X_all = cleared_data_all.drop([target_feature_name], axis=1)
data_Y_all = cleared_data_all[target_feature_name]
data_X_train_all, data_X_test_all, data_Y_train_all, data_Y_test_all = train_test_split(data_X_all, data_Y_all, test_size=test_size, random_state=seed)

data_X_only_percent = cleared_data_only_percent.drop([target_feature_name], axis=1)
data_Y_only_percent = cleared_data_only_percent[target_feature_name]
data_X_train_only_percent, data_X_test_only_percent, data_Y_train_only_percent, data_Y_test_only_percent = train_test_split(data_X_only_percent, data_Y_only_percent, test_size=test_size, random_state=seed)

datasets = [{'X_train': data_X_train_only_percent, 'X_test': data_X_test_only_percent, 'Y_train': data_Y_train_only_percent,
             'Y_test': data_Y_test_only_percent, 'name': 'only_percent_dataset'},
            {'X_train': data_X_train_all, 'X_test': data_X_test_all,
             'Y_train': data_Y_train_all, 'Y_test':data_Y_test_all, 'name': 'original_data'}]

SVR model

Define grid search params for SVR

In [5]:
SVR_greed_search_params = {
    'C': [i for i in np.linspace(0.1, 100, num=20)],
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': [i for i in range(2, 20)],
    'gamma' : ['scale', 'auto']
}

In [6]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    svr_model = SVR()
    grid_search = GridSearchCV(svr_model, SVR_greed_search_params, n_jobs=7)
    grid_search.fit(dataset.get("X_train"), dataset.get("Y_train"))

    best_params = grid_search.best_params_
    print(f'Best hyper parameters: {best_params}')

    best_svr_model = SVR(
        C=best_params.get('C'),
        kernel=best_params.get('kernel'),
        degree=best_params.get('degree'),
        gamma=best_params.get('gamma')
    )
    best_svr_model.fit(dataset.get("X_train"), dataset.get("Y_train"))

    test_predict = best_svr_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print(f'RMSE for SVR: ', rmse)
    print(f'R2_score for SVR: ', r2)
    print()

dataset: only_percent_dataset
Best hyper parameters: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
RMSE for SVR:  0.059422630844167774
R2_score for SVR:  0.8603970590781945

dataset: original_data
Best hyper parameters: {'C': 0.1, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
RMSE for SVR:  0.053753498402619324
R2_score for SVR:  0.9055023742080813



Random forest model

Define grid search params for Random forest model

In [7]:
random_forest_greed_search_params = {
    'n_estimators': [i for i in range(501, 1002, 250)],
    'criterion' : ["squared_error", "friedman_mse"],
}

In [8]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    random_forest_model = RandomForestRegressor()
    grid_search = GridSearchCV(random_forest_model, random_forest_greed_search_params, n_jobs=7)
    grid_search.fit(dataset.get("X_train"), dataset.get("Y_train"))

    best_params = grid_search.best_params_
    print(f'Best hyper parameters: {best_params}')

    best_random_forest_model = RandomForestRegressor(
        n_estimators=best_params.get('n_estimators'),
        criterion=best_params.get('criterion')
    )
    best_random_forest_model.fit(dataset.get("X_train"), dataset.get("Y_train"))

    test_predict = best_random_forest_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print(f'RMSE for SVR: ', rmse)
    print(f'R2_score for SVR: ', r2)

dataset: only_percent_dataset
Best hyper parameters: {'criterion': 'squared_error', 'n_estimators': 501}
RMSE for SVR:  0.044803395865270886
R2_score for SVR:  0.9206379858792111
dataset: original_data
Best hyper parameters: {'criterion': 'friedman_mse', 'n_estimators': 501}
RMSE for SVR:  0.03621971822831626
R2_score for SVR:  0.9570960218099002


Linear Regression

In [9]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    linear_model = LinearRegression()
    linear_model.fit(dataset.get("X_train"), dataset.get("Y_train"))
    test_predict = linear_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print("RMSE for Linear Regression:", rmse)
    print("R2_score for Linear Regression", r2)
    print()

dataset: only_percent_dataset
RMSE for Linear Regression: 0.05202448544945905
R2_score for Linear Regression 0.8929944254671817

dataset: original_data
RMSE for Linear Regression: 0.07341650230248918
R2_score for Linear Regression 0.8237233418607327



Polynomial Regression (Not working, memory error)

In [10]:
for dataset in datasets:
    r2_scores = []
    rmse_values = []
    print(f'dataset: {dataset.get("name")}')
    for degree in range(2, 20): #  working only with degree=2
        polynomial_features = PolynomialFeatures(degree=degree)
        X_train_poly = polynomial_features.fit_transform(dataset.get("X_train"))
        model = LinearRegression()
        model.fit(X_train_poly, dataset.get("Y_train"))

        test_predict = model.predict(polynomial_features.fit_transform(dataset.get("X_test")))
        rmse_values.append(np.sqrt(mean_squared_error(test_predict, dataset.get("Y_test"))))
        r2_scores.append(r2_score(test_predict, dataset.get("Y_test")))

    max_r2_scores_degree = r2_scores.index(max(r2_scores))
    print(f'Degree with maximum r2: {max_r2_scores_degree+2}')
    print(f"RMSE for Polynomial Regression with degree {max_r2_scores_degree+2}:", r2_scores[max_r2_scores_degree])
    print(f"R2_score for Polynomial Regression with degree {max_r2_scores_degree+2}:", rmse_values[max_r2_scores_degree])
    print()

    max_rmse_degree = rmse_values.index(max(rmse_values))
    print(f'Degree with minimum rmse: {max_rmse_degree+2}')
    print(f"RMSE for Polynomial Regression with degree {max_rmse_degree+2}:", r2_scores[max_rmse_degree])
    print(f"R2_score for Polynomial Regression with degree {max_rmse_degree+2}:", rmse_values[max_rmse_degree])
    print()

dataset: only_percent_dataset


MemoryError: Unable to allocate 570. GiB for an array with shape (1625, 47049960) and data type float64

In [11]:
tree_greed_search_params = {
    'splitter': ["best", "random"],
    'criterion' : ["squared_error", "friedman_mse", "absolute_error", "poisson"],
}

In [12]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    tree_model = DecisionTreeRegressor()

    grid_search = GridSearchCV(tree_model, tree_greed_search_params, n_jobs=7)
    grid_search.fit(dataset.get("X_train"), dataset.get("Y_train"))

    best_params = grid_search.best_params_
    print(f'Best hyper parameters: {best_params}')

    best_tree_model = DecisionTreeRegressor(
        splitter=best_params.get('splitter'),
        criterion=best_params.get('criterion')
    )

    best_tree_model.fit(dataset.get("X_train"), dataset.get("Y_train"))
    test_predict = best_tree_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print(f'RMSE for tree: ', rmse)
    print(f'R2_score for tree: ', r2)

dataset: only_percent_dataset
Best hyper parameters: {'criterion': 'friedman_mse', 'splitter': 'random'}
RMSE for tree:  0.05580004046803804
R2_score for tree:  0.8768994923312178
dataset: original_data
Best hyper parameters: {'criterion': 'squared_error', 'splitter': 'best'}
RMSE for tree:  0.057037827184225995
R2_score for tree:  0.8936020220536308


Bagging for regression models

In [13]:
best_models = [
    [
        {'model': SVR(C=0.1, degree=3, gamma='scale', kernel='poly'), 'name': 'svr_model'},
        {'model': LinearRegression(), 'name': 'linear_model'},
        {'model': DecisionTreeRegressor(criterion="friedman_mse", splitter='random'), 'name': 'tree'}
    ],
    [
        {'model': SVR(C=0.1, degree=3, gamma='scale', kernel='poly'), 'name': 'svr_model'},
        {'model': LinearRegression(), 'name': 'linear_model'},
        {'model': DecisionTreeRegressor(criterion="squared_error", splitter='best'), 'name': 'tree'}
    ]
]

for models, dataset in zip(best_models, datasets):
    print(f'dataset: {dataset.get("name")}')
    for model in models:
        bagging_model = BaggingRegressor(model.get('model'), max_samples=200, n_estimators=300)
        bagging_model.fit(dataset.get("X_train"), dataset.get("Y_train"))
        test_predict = bagging_model.predict(dataset.get("X_test"))
        rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
        r2 = r2_score(dataset.get("Y_test"), test_predict)
        print(f'RMSE for Bagging {model.get("name")}:', rmse)
        print(f'R2_score for Bagging {model.get("name")}:', r2)
        print()

dataset: only_percent_dataset
RMSE for Bagging svr_model: 0.0756793031665327
R2_score for Bagging svr_model: 0.7735642133413327

RMSE for Bagging linear_model: 0.04872251779606758
R2_score for Bagging linear_model: 0.9061465456108543

RMSE for Bagging tree: 0.0532003277993874
R2_score for Bagging tree: 0.888102745029903

dataset: original_data
RMSE for Bagging svr_model: 0.07076365151355261
R2_score for Bagging svr_model: 0.8362324324155682

RMSE for Bagging linear_model: 0.0383915687476038
R2_score for Bagging linear_model: 0.9517964364627444

RMSE for Bagging tree: 0.043905995982801975
R2_score for Bagging tree: 0.9369543545876134

